In [66]:
import os
import shutil
import dataiku
import pandas as pd
import mlflow

/Users/christelleren/DSS/design/code-envs/python/py36_mlflow/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  """


In [6]:
client = dataiku.api_client()
project = client.get_default_project()

In [13]:
# get train dataset
train_dataset = dataiku.Dataset("training_data")
evaluation_dataset = dataiku.Dataset("eval_data")

In [60]:
# get output saved model
sm = project.get_saved_model("VdHxdbkg")

# get train dataset as a pandas dataframe
df = train_dataset.get_dataframe()

In [18]:
# get the path of a local managed folder where to temporarily save the trained model
mf = dataiku.Folder("cTcsiBqc")
path = mf.get_path()

In [19]:
model_subdir = "my_subdir"
model_dir = os.path.join(path, model_subdir)

In [20]:
from sklearn.linear_model import LogisticRegression

In [54]:
y = df['y']
X = df.drop('y', axis=1)

In [42]:
job_df = pd.get_dummies(df.job, prefix='job')
marital_df = pd.get_dummies(df.marital, prefix='marital')
education_df = pd.get_dummies(df.education, prefix='education')

In [36]:
month_dic = {
    'apr': 5,
    'aug': 8,
    'dec': 12,
    'feb': 2,
    'jan': 1,
    'jul': 7,
    'jun': 6,
    'mar': 3,
    'may': 5,
    'nov': 11,
    'oct': 10,
    'sep': 9}

In [63]:
X.drop(['job', 'marital', 'education'], axis=1, inplace=True)
X = pd.concat([X, job_df, marital_df, education_df], axis=1)
X.month.replace(month_dic, inplace=True)
X.replace({"no": 0, "yes": 1}, inplace=True)
clf = LogisticRegression(random_state=0).fit(X, y)

In [71]:
if os.path.exists(model_dir):
    shutil.rmtree(model_dir)

try:
    # ...train your model...
    clf = LogisticRegression(random_state=0).fit(X, y)
    
    # ...save it with package specific MLflow method (here, SKlearn)...
    mlflow.sklearn.save_model(clf, model_dir)

    # import the model, creating a new version
    mlflow_version = sm.import_mlflow_version_from_managed_folder("v03", "cTcsiBqc", model_subdir, "py36_mlflow")
finally:
    shutil.rmtree(model_dir)

/Users/christelleren/DSS/design/code-envs/python/py36_mlflow/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [73]:
# setting metadata (target name, classes,...)
mlflow_version.set_core_metadata(target_column_name="y",
                             class_labels=["no", "yes"],
                             get_features_from_dataset="eval_data")

In [74]:
# evaluate the performance of this new version, to populate the performance screens of the saved model version in DSS
mlflow_version.evaluate("eval_data")

DataikuException: com.dataiku.dip.io.SocketBlockLinkKernelException: Could not run command EVALUATE: : <class 'ValueError'> : could not convert string to float: 'blue-collar'